# The Battle of Neighborhoods Zapopan vs Tlajomulco


## 1. Introduction

Jalisco in Mexico is one of its most important cities, mainly known by Tequila and Mariachis. This great city is located on the west side of Mexico and given its great location, big technology companies established branches that produce software and hardware not only for the LATAM region but for the world. Given this situation now this city is known as the LATAM Sillicon Valley. This condition for the city produce a lot of business and allows to increase the region's economy and nowadays Guadalajara is the third economy of Mexico.

Reviewing the main neighborhoods in Guadalajara, the biggest one for ecomonics reasons is in Zapopan Municipality, but the growing of Tlajomulco Municipality is the the one that is growing at the fastest rate. this project pretends to analize information of both Municipalities in order to propose a business oportunity for the Tlajomulco Municipality.


<img src = "https://www.jalisco.gob.mx/sites/default/files/archivos-municipios/mapa_mex.png" width=500>

<img src = "https://zonaguadalajara.com/zona-metropolitana-de-guadalajara/zonagdl_mapa/" width=500>

## 2. Business Problem


As I mentioned in the previous section, Zapopan is the Municipality reference for Guadalajara city given its economic weight. For this project we use Zapopan geolocation data as benchmark and try to find a business opportunity. 

In concrete I will compare both Municipalities classified as B or B+ (meaning medium class, medium plus class), and Zapopan has a lot of sectors belonging to that classification and I will try to check in Tlajomulcos South (in this part of the city we have A, B and B+ neighborhoods) if there is missing a business that is a frecquent business for this kind of market. Initial field research shows that there is missing Sport Bars, but we will review the information in order to confirm if there is an opportunity to establish an Sport Bar in the south of the city in Tlajomulco Municipality.

<img src = "https://images.sipse.com/HNi0o5JnA5TAbyK257WPi-EWgJw=/1654x1016/smart/imgs/022016/0602163e0b7916c.jpg" width=500>

## 3. Data

The information that I will use in order to get the Post Code information is given by SEPOMEX (National Post Provider) and I will use only the information of Zapopan and Tlajomulco Municipality and I will analize the information as we review in previous week with Foursquare.

### Glossary

- Código Postal: Postal Code
- Asentamiento: Neighborhood
- Tipo de Asentamiento: Neighborhood Type
- Municipio: Borough
- Estado: State
- Ciudad: City
- Latitud: Latitude
- Longitud: Longitude

Following you will find the data that I will use for this project

In [32]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0



df_data_0 = pd.read_excel(body)
df_data_0.head()


,Código Postal,Asentamiento,Tipo de Asentamiento,Municipio,Estado,Ciudad,latitud,longitud
0,45640,Rinconadas del Sol,Fraccionamiento,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
1,45640,El Cortijo,Ejido,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
2,45640,Vista Sur Residencial,Fraccionamiento,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
3,45640,Valle de Santa Cruz,Colonia,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
4,45640,Buenavista,Colonia,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942


I will put all the Neighborhood in the correspondant Postal Code

In [33]:
#Combine neighbourhood with the same Post Code
df_group = df_data_0[['Código Postal','Asentamiento','Municipio']]
df_group = df_group.groupby(['Código Postal','Municipio'])['Asentamiento'].apply(list)
df_group = df_group.sample(frac=1).reset_index()
df_group['Asentamiento']=df_group['Asentamiento'].str.join(',')
df_group.head()

,Código Postal,Municipio,Asentamiento
0,45076,Zapopan,Unidad Cadete Francisco Márquez
1,45675,Tlajomulco de Zúñiga,"Los Silos,Los Sauces,La Purísima,El Lago,El Ca..."
2,45158,Zapopan,"Los Laureles,San Gilberto,Prados de San Ignaci..."
3,45014,Zapopan,Villa Panamericana
4,45070,Zapopan,"Arboledas 1a Secc,Arboledas 2a Secc,El Colli E..."


## 4. Methodology

In order to evaluate the information I will use geolocation data given by the Mexican Postal Office (SEPOMEX), and I will use Foursquare Development API. The methodology will be BENCHMARK as follows:

1. Review the information corresponding to Zapopan Borough, because that Borough will be the Benchmark to extrappolate the information to Tlajomulco Borough.
2. Use Foursquare API to identify the business that are open in Zapopan, and after that centrate the efforts to Sports Bars in order to detect the density of that business in the area.
3. Analiazing the information using K-means to cluster information and detect areas in the Borough with that kind of business.
4. Do the same analysis with Tlajomulco Borough (Foursquare analysis with geolocation data and cluster anlysis using k-means method)
5. Confront results with Zapopan, to evaluate if there is an oppportunity to establish an Sport Bar in that Borough. 

## 5. Analysis

First I review the Zapopan information

In [34]:
#importing libraries
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Use geopy library to get the latitude and longitude values of Zapopan, Jal.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent zp_explorer, as shown below.

In [61]:
address = 'Zapopan, Jal'

geolocator = Nominatim(user_agent="zp_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zapopan City are {}, {}.'.format(latitude, longitude))

zapopan_data = df_data_0[df_data_0['Municipio'] == 'Zapopan'].reset_index(drop=True)
zapopan_data.head()


The geograpical coordinate of Zapopan City are 20.7032055, -103.4261243.


,Código Postal,Asentamiento,Tipo de Asentamiento,Municipio,Estado,Ciudad,latitud,longitud
0,45010,Paseos del Parque,Fraccionamiento,Zapopan,Jalisco,Zapopan,20.6805,-103.4491
1,45010,Ciudad Granja,Colonia,Zapopan,Jalisco,Zapopan,20.6805,-103.4491
2,45010,Chapalita Inn,Colonia,Zapopan,Jalisco,Zapopan,20.6805,-103.4491
3,45010,Lomas Del Colli,Fraccionamiento,Zapopan,Jalisco,Zapopan,20.6805,-103.4491
4,45010,Coto Del Rey,Fraccionamiento,Zapopan,Jalisco,Zapopan,20.6805,-103.4491


Use geopy library to get the latitude and longitude values of Tlajomulco, Jal. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent tj_explorer, as shown below.

In [36]:
address = 'Tlajomulco de Zuñiga, Jal'

geolocator = Nominatim(user_agent="tj_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tlajomulco City are {}, {}.'.format(latitude, longitude))

tlajomulco_data = df_data_0[df_data_0['Municipio'] == 'Tlajomulco de Zúñiga'].reset_index(drop=True)
tlajomulco_data.head()

The geograpical coordinate of Tlajomulco City are 20.4737273, -103.4469713.


,Código Postal,Asentamiento,Tipo de Asentamiento,Municipio,Estado,Ciudad,latitud,longitud
0,45640,Rinconadas del Sol,Fraccionamiento,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
1,45640,El Cortijo,Ejido,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
2,45640,Vista Sur Residencial,Fraccionamiento,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
3,45640,Valle de Santa Cruz,Colonia,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942
4,45640,Buenavista,Colonia,Tlajomulco de Zúñiga,Jalisco,Tlajomulco de Zúñiga,20.5008,-103.4942


Let's continue with the creation of the map

- Zapopan data will be in blue
- Tlajomulco data will be in red

In [37]:
# create map of Zapopan using latitude and longitude values
map_zmg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(zapopan_data['latitud'], zapopan_data['longitud'], zapopan_data['Municipio'], zapopan_data['Asentamiento']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zmg)  
for lat, lng, borough, neighborhood in zip(tlajomulco_data['latitud'], tlajomulco_data['longitud'], tlajomulco_data['Municipio'], tlajomulco_data['Asentamiento']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#2186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zmg) 
    
map_zmg

As you can see on the map, the idea of this analysis is to see if there is an opportunity to establishes an Sports Bar in the SOUTH of the city

### Using Foursquare API


In [38]:

CLIENT_ID = 'EU0UYEMIJL41HG5HF1PJ4DLNPVRPH0RFK1WEL1A1E1B2D5QE' # your Foursquare ID
CLIENT_SECRET = 'OA4PW4R4KDNPUWRDU3T4PNVW5EIIKUZAPN1CUZX3FUG41BLA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EU0UYEMIJL41HG5HF1PJ4DLNPVRPH0RFK1WEL1A1E1B2D5QE
CLIENT_SECRET:OA4PW4R4KDNPUWRDU3T4PNVW5EIIKUZAPN1CUZX3FUG41BLA


### Explore Neighborhoods in Zapopan

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called zapopan_venues

In [40]:
zapopan_venues = getNearbyVenues(names=zapopan_data['Asentamiento'],
                                   latitudes=zapopan_data['latitud'],
                                   longitudes=zapopan_data['longitud']
                                  )

Paseos del Parque
Ciudad Granja
Chapalita Inn
Lomas Del Colli
Coto Del Rey
Glorias Del Colli
La Castilla
Cumbres
Jardines de Chapalita
Bellavista
Volcán Del Colli
Vistas Del Pinar
Rinconada Del Parque
EXITMEX
Álamos
Ciudad Judicial Federal
Ayamonte
Villa Panamericana
Centro de Investigación y de Estudios Avanzados
Paseos Universidad
Lomas Universidad
Villas de Vallarta
Jocotán
El Bajío
Puertas Del Tule
Santa Maria Del Pueblito
Residencial Del Parque
Santillana Parque Residencial
Santa Catalina
Los Jazmines
San Juan de Ocotán
Valle Real
La Magdalena
La Mojonera
Las Higuerillas
Porta Real
Solares
Camichines Vallarta
Eucalipto Vallarta
Prados Vallarta
Real Vallarta
Vallarta La Patria
Jardines Vallarta
Rinconada de los Novelistas
El Carmen Hábitat
Parque Sereno
Atlas Chapalita
Chapalita de Occidente
Chapalita las Fuentes
Guadalupe Jardín
Jardines de Guadalupe
Prados de Guadalupe
La Estancia
Plaza Guadalupe
Rinconada Guadalupe
Plaza Guadalupe 2a Sección
Nuestra Señora de Guadalupe
Los Olivo

### Let's check the size of the resulting dataframe


In [41]:
print(zapopan_venues.shape)
zapopan_venues.head()

(4860, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paseos del Parque,20.6805,-103.4491,Tacos Los Amigos,20.683433,-103.450212,Taco Place
1,Paseos del Parque,20.6805,-103.4491,Casa de Letras,20.680203,-103.448118,Bookstore
2,Paseos del Parque,20.6805,-103.4491,Starbucks,20.682384,-103.445837,Coffee Shop
3,Paseos del Parque,20.6805,-103.4491,Taquería Kalitos,20.680557,-103.445140,Taco Place
4,Paseos del Parque,20.6805,-103.4491,Dos de azúcar,20.681561,-103.445532,Coffee Shop


### Lets see how many bars are in Zapopan

In [42]:
zapopanbar_data = zapopan_venues[zapopan_venues['Venue Category'] == 'Bar'].reset_index(drop=True)
print(zapopanbar_data.shape)
zapopanbar_data

(29, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agua Fría,20.6697,-103.3603,Coyote Rojo,20.672656,-103.358611,Bar
1,Agua Fría,20.6697,-103.3603,Modi,20.670488,-103.357159,Bar
2,Arroyo Hondo,20.6697,-103.3603,Coyote Rojo,20.672656,-103.358611,Bar
3,Arroyo Hondo,20.6697,-103.3603,Modi,20.670488,-103.357159,Bar
4,Balcones de La Cantera,20.6697,-103.3603,Coyote Rojo,20.672656,-103.358611,Bar
5,Balcones de La Cantera,20.6697,-103.3603,Modi,20.670488,-103.357159,Bar
6,Miguel Hidalgo,20.6697,-103.3603,Coyote Rojo,20.672656,-103.358611,Bar
7,Miguel Hidalgo,20.6697,-103.3603,Modi,20.670488,-103.357159,Bar
8,Arroyo Hondo 2a Secc,20.6697,-103.3603,Coyote Rojo,20.672656,-103.358611,Bar
9,Arroyo Hondo 2a Secc,20.6697,-103.3603,Modi,20.670488,-103.357159,Bar


### Now how many Sports Bar are in Zapopan

In [43]:
zapopansports = zapopan_venues[zapopan_venues['Venue Category'] == 'Sports Bar'].reset_index(drop=True)
print(zapopansports.shape)
zapopansports

(24, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paseos del Parque,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
1,Ciudad Granja,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
2,Chapalita Inn,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
3,Lomas Del Colli,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
4,Coto Del Rey,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
5,Glorias Del Colli,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
6,La Castilla,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
7,Cumbres,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
8,Jardines de Chapalita,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar
9,Bellavista,20.6805,-103.4491,Wingman,20.682403,-103.445814,Sports Bar


### Let's see in the map the location of the Bars and Sports Bars in Zapopan


In [44]:
# create map of Zapopan using latitude and longitude values
map_zapopan = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(zapopanbar_data['Venue Latitude'], zapopanbar_data['Venue Longitude'], zapopan_data['Municipio'], zapopanbar_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zapopan)  
for lat, lng, borough, neighborhood in zip(zapopansports['Venue Latitude'], zapopansports['Venue Longitude'], zapopan_data['Municipio'], zapopansports['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#2186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zapopan) 
    
map_zapopan

## With this previous analysis is evident that there not enogh data registered in Fourquare regarding Bars (in reality there a lot of bars in Zapopan area), therefore I will review in Zapopan how many venues categories are and more important whats is the most common category in this Borough in order to proceed with my analysis.

In [45]:
print('There are {} uniques categories.'.format(len(zapopan_venues['Venue Category'].unique())))

There are 93 uniques categories.


In [46]:
zapopan_venues.groupby('Venue Category').count()


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Art Gallery,7,7,7,7,7,7
Athletics & Sports,38,38,38,38,38,38
BBQ Joint,3,3,3,3,3,3
Bakery,53,53,53,53,53,53
Bar,29,29,29,29,29,29
Beach Bar,75,75,75,75,75,75
Bed & Breakfast,7,7,7,7,7,7
Bistro,17,17,17,17,17,17
Bookstore,31,31,31,31,31,31


## Given the above information I will make the analysis with TACO PLACES

In [47]:
zapopantaco = zapopan_venues[zapopan_venues['Venue Category'] == 'Taco Place'].reset_index(drop=True)
print(zapopantaco.shape)
zapopantaco.head()

(596, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paseos del Parque,20.6805,-103.4491,Tacos Los Amigos,20.683433,-103.450212,Taco Place
1,Paseos del Parque,20.6805,-103.4491,Taquería Kalitos,20.680557,-103.445140,Taco Place
2,Paseos del Parque,20.6805,-103.4491,Tacos Joel y Angélica,20.683639,-103.447239,Taco Place
3,Paseos del Parque,20.6805,-103.4491,Tacos La Michoacana,20.683436,-103.447150,Taco Place
4,Paseos del Parque,20.6805,-103.4491,Tacos Joel,20.684384,-103.447104,Taco Place


In [48]:
# create map of Zapopan using latitude and longitude values
map_tacozapopan = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(zapopantaco['Venue Latitude'], zapopantaco['Venue Longitude'], zapopan_data['Municipio'], zapopantaco['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tacozapopan)  

map_tacozapopan

In [49]:
from folium import plugins

# let's start again with a clean copy of the map of Zapopan
map_tacozapopan = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_tacozapopan)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(zapopantaco['Venue Latitude'], zapopantaco['Venue Longitude'], zapopantaco['Venue Category']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
map_tacozapopan

## As we can see in the map we have 7 cluster that has 596 Taco places, with the following information:

- The cluster with more Taco places has 172
- The cluster with mimimum Taco places has 25
- The average is 85 places per cluster

## Lets compare with Tlajomulco

In [50]:
tlajomulco_venues = getNearbyVenues(names=tlajomulco_data['Asentamiento'],
                                   latitudes=tlajomulco_data['latitud'],
                                   longitudes=tlajomulco_data['longitud']
                                  )

Rinconadas del Sol
El Cortijo
Vista Sur Residencial
Valle de Santa Cruz
Buenavista
Santa Cruz de las Flores
Tlajomulco Centro
Lomas de Santa Anita
Jardines de Tlajomulco
Huertas Productivas de Jalisco
El Tecolote
La Teja
Cima Del Sol
La Providencia
Las Víboras (Fraccionamiento Valle de las Flores)
Banús
Colinas de Santa Anita
Cofradía
La Lomita
Larios
Las Flores
La Purísima
Héroes de Nacozari
Prados de La Higuera
Las Galeanas
Los Mezquites
Mirador del Valle
Los Álamos
Arcos de la Cruz
Bosques de la Esperanza
Colinas Desarrollo
Rancho el Derramadero
Rancho el Zapote
Hacienda Real
Rancho los Ocampo
Los Pinos
Nuevo Tlajomulco
Patria
Rinconada las Auroras
Valle de Tlajomulco
Villanova
El Esquivel
Lomas del Pedregal
Valle de los Emperadores
Lomas de Tejeda
El Mirador
Lomas Habitacional
Residencial Lomas de Tejeda
Tierra de la Esperanza
Los Tapia
Hacienda La Noria
Rinconada de los Nogales
El Palomar
Cielo Contry Club
Las Villas
San Martin Del Tajo
Soledad de Cruz Vieja
Santa Cruz de la Loma 

## Calculating the taco places in Tlajomulco

In [51]:
tlajomulcotaco = tlajomulco_venues[tlajomulco_venues['Venue Category'] == 'Taco Place'].reset_index(drop=True)
print(tlajomulcotaco.shape)
tlajomulcotaco.head()

(54, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tulipanes,20.5817,-103.4406,Tacos El Pueblo,20.584786,-103.440517,Taco Place
1,Tulipanes,20.5817,-103.4406,Tacos El Primo Y El Felino,20.584870,-103.440743,Taco Place
2,La Tijera,20.5817,-103.4406,Tacos El Pueblo,20.584786,-103.440517,Taco Place
3,La Tijera,20.5817,-103.4406,Tacos El Primo Y El Felino,20.584870,-103.440743,Taco Place
4,Hacienda La Tijera,20.5817,-103.4406,Tacos El Pueblo,20.584786,-103.440517,Taco Place


## Let see them on the map

In [52]:
# create map of Zapopan using latitude and longitude values
map_tacotlajomulco = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tlajomulcotaco['Venue Latitude'], tlajomulcotaco['Venue Longitude'], tlajomulco_data['Municipio'], tlajomulcotaco['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tacotlajomulco)  

map_tacotlajomulco

In [54]:
from folium import plugins

# let's start again with a clean copy of the map of Zapopan
map_tacotlajomulco = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_tacotlajomulco)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(tlajomulcotaco['Venue Latitude'], tlajomulcotaco['Venue Longitude'], tlajomulcotaco['Venue Category']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
map_tacotlajomulco

## As we can see in the map we have 3 cluster that has 54 Taco places, with the following information:

- The cluster with more Taco places has 25
- The cluster with mimimum Taco places has 13
- The average is 18 places per cluster

## Let see them together

In [58]:
# create map of ZMG using latitude and longitude values
taco_zmg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(zapopantaco['Venue Latitude'], zapopantaco['Venue Longitude'], zapopan_data['Municipio'], zapopantaco['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(taco_zmg)  
for lat, lng, borough, neighborhood in zip(tlajomulcotaco['Venue Latitude'], tlajomulcotaco['Venue Longitude'], tlajomulco_data['Municipio'], tlajomulcotaco['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#2186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(taco_zmg) 
    
taco_zmg

## Let's compare how many Neighborhoods has every Borough

In [62]:
print(zapopan_data.shape)
print(tlajomulco_data.shape)

(613, 8)
(225, 8)


- Zapopan has 613 neighborhoods
- Tlajomulco de Zuñiga has 225 neighborhoods

## To continue the benchmark analysis using Zapopan Borough, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [63]:
# one hot encoding
zapopan_onehot = pd.get_dummies(zapopan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zapopan_onehot['Neighborhood'] = zapopan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zapopan_onehot.columns[-1]] + list(zapopan_onehot.columns[:-1])
zapopan_onehot = zapopan_onehot[fixed_columns]

zapopan_onehot.head()

,Wings Joint,Art Gallery,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach Bar,Bed & Breakfast,Bistro,Bookstore,...,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Theme Park,Tourist Information Center,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
zapopan_grouped = zapopan_onehot.groupby('Neighborhood').mean().reset_index()
zapopan_grouped

,Neighborhood,Wings Joint,Art Gallery,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach Bar,Bed & Breakfast,Bistro,...,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Theme Park,Tourist Information Center,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant
0,12 de Diciembre,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.142857,0.142857,0.0,0.0,0.000000,0.000000
1,Agraria Río Blanco,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,Agrícola,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,Agua Blanca,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,Agua Blanca Industrial,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5,Agua Blanca Sur,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.071429,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
6,Agua Fría,0.000000,0.022222,0.000000,0.0,0.044444,0.044444,0.00,0.022222,0.022222,...,0.000000,0.088889,0.066667,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.022222
7,Al Viento Residencial I,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,...,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
8,Alianza de Cazadores Diana,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.100000,0.000000
9,Altabrisa,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,...,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = zapopan_grouped['Neighborhood']

for ind in np.arange(zapopan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(zapopan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,12 de Diciembre,Hotel,Tourist Information Center,Theme Park,Convenience Store,Coffee Shop
1,Agraria Río Blanco,Convenience Store,Restaurant,Athletics & Sports,Taco Place,Park
2,Agrícola,Soccer Field,Food Truck,Mexican Restaurant,Drugstore,Grocery Store
3,Agua Blanca,Soccer Field,Food Truck,Mexican Restaurant,Drugstore,Grocery Store
4,Agua Blanca Industrial,Soccer Field,Food Truck,Mexican Restaurant,Drugstore,Grocery Store


In [81]:
print(neighborhoods_venues_sorted.shape)

(486, 6)


## Let's see how many times Taco place venue is the 1st most common venue

In [82]:
tacovenue = neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'] == 'Taco Place'].reset_index(drop=True)
print(tacovenue.shape)

(152, 6)


## Let's see how many times Taco place venue is the 2nd most common venue

In [83]:
tacovenue = neighborhoods_venues_sorted[neighborhoods_venues_sorted['2nd Most Common Venue'] == 'Taco Place'].reset_index(drop=True)
print(tacovenue.shape)

(16, 6)


## Let's see how many times Taco place venue is the 3rd most common venue

In [84]:
tacovenue = neighborhoods_venues_sorted[neighborhoods_venues_sorted['3rd Most Common Venue'] == 'Taco Place'].reset_index(drop=True)
print(tacovenue.shape)

(0, 6)


## Let's see how many times Taco place venue is the 4th most common venue

In [80]:
tacovenue = neighborhoods_venues_sorted[neighborhoods_venues_sorted['4th Most Common Venue'] == 'Taco Place'].reset_index(drop=True)
print(tacovenue.shape)

(27, 6)


## Let's see how many times Taco place venue is the 5th most common venue

In [85]:
tacovenue = neighborhoods_venues_sorted[neighborhoods_venues_sorted['5th Most Common Venue'] == 'Taco Place'].reset_index(drop=True)
print(tacovenue.shape)

(1, 6)


## 6. Results

1. The first important result is that THERE is NOT enough data to establish if it is a good business to put an Sports Bar either Zapopan or Tlajomulco de Zuñiga because there are not good enough business registered in Foursquare. However on field we can see there are a growing demand for that kind of business specially in Tlajomulco de Zuñiga, I suggest to go deeper in market research if one person decide to put an Sports Bar in Tlajomulco de Zuñiga. By going deeper, I mean to use another source different to Foursquare, in order to see if it is a good idea to put an Sports Bar in the South og Guadalajara, using as benchmark the north borough: Zapopan. If for some reason decide to establish an Sports Bar in Tlajomulco I recommend to use Foursquere as business diferential, because as preeliminary analysis I found that not many Sports Bar registered in the app.

2. Zapopan is a good benchmark to use as reference in Tlajomulco because according to INEGI data (Mexican Statistical Government Office), represent the 26% of the Metropolitan Guadalajara population with 1,243,756 habitants in its 613 Neighborhoods.

3. Tlajomulco de Zuñiga represents according to Inegi Data the 9% of the Metropolitan Guadalajara population with 416,626 habitants in 225 Neighborhoods.

4. Using Zapopan as Benchmark we can see that Tlajomulco de zuñiga in population represents the 33.5% of Zapopan Borough.

5. Comparing Neighborhoods, Tlajomulco de Zuñiga represents the 36.7% og Zapopan's Neighborhoods.

6. After the first analysis of the data that we have from Foursquare, I decided to review deeper the information in order to see one of the more attractive business in the area adn with this esploratory analysis if Zapopans data it results as good option for analysis the TACO PLACES. 

7. Reviewing Zapopan information regarding the venues it results that in the area were 596 Taco places that represents the 12.3% of the venues of the area, but the relevant information is the following:

- From the total neighborhood registered in Foursquare the Taco Places represents the 31.3% has tha 1st Most Common Venue
- From the total neighborhhod registered in Foursquare the Taco Places represents the 40.3% of the venues that are inside of the top 5 common venues
- Given the above reasons Taco places is a good option as business in the area

8. Using geospacial analysis it results 7 clusters in Zapopan area with opportunity to develop some clusters because the diferental between the maximum value and the minimum value is 147 places, where the standard deviation is 58.5, therefore there are opportunity to develop in the clusters with less venues. Additional data will be useful in order to see if there optimal to establish a Taco Place in the cluster with lower Taco PLaces because we saw as first finding in the analysis that not many busineess use Foursquares as business diferential.

9. Reviewing Tlajomulco de Zuñiga data it results that only 54 venues appears as Taco Places. If we consider that Tlajomulco represent around the third part of Zapopan then the potential to put a Taco Place in Tlajomulco de Zuñiga is strong. Therefore given the analysis I strongly recommend to put a Taco Place in Tlajomulco de Zuñiga area. Only the 10 % of the market is develop in the area and remember that for Zapopan Area (that is our benchmark) the Taco places venue, represents the 40.3% of the venues inside of the Top 5 Common Venues.


## 7. Discussion

I strongly recommend to go further analysis with additional data different to Foursquare because as I reviewed the information it results that not many business registered its bussiness in this application. In Mexico the information that can be useful would be INEGI data to complement this analysis.

## Conclussions:

1. In Guadalajara not many business use Foursquare to take advantage of this application, therefore the business that use this platform has and advantage to the ones that are not using it.
2. The initial approach to consider Zapopan as Benchmark of Tlajomulco de Zuñiga borough was a good decision, becuase it represents almost the 30% os the Metropolitan Guadalajara city.
3. The initial approach to consider Sports Bar to suggest as a business opportunity in Tlajomulco Area using Foursquare data was declined after the exploratory analysis, adn it results in change the business opportunity area to Taco places
4. Taco places represents a good business opportunity to develop in Tlajomulco de Zuñiga area, because is a highly appreciated venue in our benchmark.
5. Inside the benchmar area (Zapopan), it represents also a good opportunity to establish a Taco place, because there are some areas that are not covered on the same rate.